In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import tsai
from tsai.all import *
from joblib import Parallel, delayed

In [3]:
data_root = '/media/scottcha/E1/Data/OAPMLData/'
ml_path = data_root + '/5.MLData/'
num_features = 978
interpolation = 1
file_label = 'co_Day1DangerAboveTreeline_small'
label = 'Day1DangerAboveTreeline'

In [4]:
def calculate_feature_mean(feature):
    print('On feature: ' + str(feature))
    return np.nanmean(X[0:5000,feature,:])

def calculate_feature_std(feature):
    print('On feature: ' + str(feature))
    return np.nanstd(X[0:5000,feature,:])

#method to standardize each batch while also replacing any nans with the mean value before standarization
class TSStandardizeNanMeanReplaceNan(Transform):
    "Standardize/destd batch of `NumpyTensor` or `TSTensor`"
    parameters, order = L('mean', 'std'), 99
    def __init__(self, mean=None, std=None, by_sample=False, by_var=False, verbose=False):
        self.mean = tensor(mean) if mean is not None else None
        self.std = tensor(std) if std is not None else None
        self.by_sample, self.by_var = by_sample, by_var
        if by_sample and by_var: self.axes = (2)
        elif by_sample: self.axes = (1, 2)
        elif by_var: self.axes = (0, 2)
        else: self.axes = ()
        self.verbose = verbose

    @classmethod
    def from_stats(cls, mean, std): return cls(mean, std)

    def setups(self, dl: DataLoader):
        if self.mean is None or self.std is None:
            pv(f'{self.__class__.__name__} setup mean={self.mean}, std={self.std}, by_sample={self.by_sample}, by_var={self.by_var}', self.verbose)
            x, *_ = dl.one_batch()
            x = torch.where(torch.isnan(x), torch.zeros_like(x), x)
            self.mean, self.std = x.mean(self.axes, keepdim=self.axes!=()), x.std(self.axes, keepdim=self.axes!=()) + 1e-7
            pv(f'mean: {self.mean}  std: {self.std}\n', self.verbose)

    def encodes(self, x:(NumpyTensor, TSTensor)):
        fill_values = torch.zeros_like(x)
        std_values = torch.zeros_like(x)       
        for i in range(0,x.shape[1]):
            fill_values[:,i,:] = torch.full_like(x[:,i,:], feature_means[i])
            std_values[:,i,:] = torch.full_like(x[:,i,:], feature_std[i])
        
        x = torch.where(torch.isnan(x), fill_values, x)
       
        if self.by_sample:        
            self.mean, self.std = x.mean(self.axes, keepdim=self.axes!=()), x.std(self.axes, keepdim=self.axes!=()) + 1e-7
            
        t = (x - fill_values) / std_values
        del fill_values, std_values
        return torch.where(torch.isnan(t), torch.zeros_like(t), t)

In [5]:
means_fn = ml_path + '/feature_means_interpolation' + str(interpolation) + '_' + file_label + 'x.npy'
std_fn = ml_path + '/feature_std_interpolation' + str(interpolation)  + '_' + file_label +   'x.npy'

In [6]:
#load the X train data from numpy as memmapped file
X = np.load(ml_path + '/Xtrain_batch_0_' + file_label + '_on_disk.npy', mmap_mode='r')

In [7]:
#if we don't have cached versions of these we need to calcualte this for the feature
#standardization, this takes awhile and requires a lot of memory so I only do it on a subset
#of the data (first 100000 rows)
feature_means = Parallel(n_jobs=4)(map(delayed(calculate_feature_mean), range(0,num_features)))
feature_std = Parallel(n_jobs=4)(map(delayed(calculate_feature_std), range(0,num_features)))

In [8]:
#cache the values
np.save(means_fn, np.asarray(feature_means))
np.save(std_fn, np.asarray(feature_std))

In [6]:
#load the values
feature_means = np.load(means_fn)
feature_std = np.load(std_fn)

In [7]:
#load the full X datafile which has both Train and Test concated
X = np.load(ml_path + '/X_all_' + file_label + '.npy', mmap_mode='r')

In [8]:
X = X[:,:,-30:]

In [9]:
#read in the corresponding label files and concat them
#can get the right values here based on the contents of hte ml_path directory
i=0


y_df = None

df = pd.read_parquet(ml_path + '/y_train_batch_' + str(i) + '_' + file_label + '.parquet')  
y_df = pd.concat([y_df, df])

df = pd.read_parquet(ml_path + '/y_test_batch_' + str(i) + '_' + file_label + '.parquet')  
y_df = pd.concat([y_df, df])

y_df = y_df.reset_index(drop=True)

In [10]:
y_df_filtered = y_df[y_df[label].isin(['Low', 'Moderate', 'Considerable', 'High'])]
#convert the labels to encoded values
from pandas.api.types import CategoricalDtype
cat_type = CategoricalDtype(categories=["Low", "Moderate", "Considerable", 'High'], ordered=True)
y_df[label + '_Cat'] = y_df[label].astype(cat_type)
y_train_df = y_df_filtered[y_df_filtered['parsed_date'] < np.datetime64('2018-11-01')]
y_test_df = y_df_filtered[y_df_filtered['parsed_date'] >= np.datetime64('2018-11-01')]
y = y_df[label + '_Cat'].cat.codes.values

dict( enumerate(y_df[label + '_Cat'].cat.categories ) )

{0: 'Low', 1: 'Moderate', 2: 'Considerable', 3: 'High'}

In [11]:
y_df.head()


,sample,latitude,longitude,UnifiedRegion,UnifiedRegionleft,Unnamed: 0,BottomLineSummary,Cornices_Likelihood,Cornices_MaximumSize,Cornices_MinimumSize,...,WindSlab_OctagonNearTreelineSouthEast,WindSlab_OctagonNearTreelineSouthWest,WindSlab_OctagonNearTreelineWest,image_paths,image_types,image_urls,rose_url,parsed_date,season,Day1DangerAboveTreeline_Cat
0,2015-11-14 00:00:00: Grand Mesa Zone,39.00,-107.50,Grand Mesa Zone,Grand Mesa Zone,16686,no-data,no-data,no-data,no-data,...,no-data,no-data,no-data,no-data,no-data,no-data,no-data,2015-11-14,15-16,NaN
1,2015-11-14 00:00:00: Grand Mesa Zone,39.25,-108.25,Grand Mesa Zone,Grand Mesa Zone,16686,no-data,no-data,no-data,no-data,...,no-data,no-data,no-data,no-data,no-data,no-data,no-data,2015-11-14,15-16,NaN
2,2015-11-14 00:00:00: Gunnison Zone,38.75,-107.50,Gunnison Zone,Gunnison Zone,16688,no-data,no-data,no-data,no-data,...,no-data,no-data,no-data,no-data,no-data,no-data,no-data,2015-11-14,15-16,Moderate
3,2015-11-15 00:00:00: Grand Mesa Zone,39.25,-108.00,Grand Mesa Zone,Grand Mesa Zone,16698,no-data,no-data,no-data,no-data,...,no-data,no-data,no-data,no-data,no-data,no-data,no-data,2015-11-15,15-16,NaN
4,2015-11-15 00:00:00: Grand Mesa Zone,39.25,-107.75,Grand Mesa Zone,Grand Mesa Zone,16698,no-data,no-data,no-data,no-data,...,no-data,no-data,no-data,no-data,no-data,no-data,no-data,2015-11-15,15-16,NaN


In [12]:
#index file which indicates which rows in X are train or test
#be carful these don't overlap

#can use a smaller train subset to make development faster
splits = (L([i for i in y_train_df.index]).shuffle(), L([i for i in y_test_df.index]).shuffle())

In [13]:
splits

((#3988) [2274,1472,4147,2940,3391,1820,1680,4920,3195,1324...],
 (#800) [5207,5440,5517,5691,5343,5898,5301,5621,5027,5135...])

In [14]:
#load and check the means and std deviations
feature_means = (np.nan_to_num(feature_means))
assert np.isnan(feature_means).any() == False
feature_std = (np.nan_to_num(feature_std, nan=1.0))
assert np.isnan(feature_std).any() == False

In [15]:
tfms = [None, [Categorize()]]
batch_tfms=[TSStandardizeNanMeanReplaceNan()]

In [16]:
X.shape

(6000, 978, 30)

In [17]:
X = X[splits[0]+splits[1]]
y = y[splits[0]+splits[1]]

In [18]:
#index file which indicates which rows in X are train or test
#be carful these don't overlap
train_test_split = 3988
num_y = 800
#can use a smaller train subset to make development faster
splits_2 = (L([i for i in range(0,train_test_split)]).shuffle(), L([i for i in range(train_test_split,train_test_split+num_y)]).shuffle())

In [19]:
splits_2

((#3988) [1016,2725,2947,932,3533,604,2131,785,2132,3453...],
 (#800) [4767,4544,4003,4341,4143,4217,4616,3996,4308,4774...])

In [20]:
X.shape

(4788, 978, 30)

In [21]:
y.shape

(4788,)

In [22]:
dls = get_ts_dls(X, y, splits=splits_2, tfms=tfms, shuffle_train=False, batch_tfms=batch_tfms)

In [23]:
model = build_ts_model(ROCKET, dls=dls) 

In [24]:
X_train, y_train = create_rocket_features(dls.train, model)
X_valid, y_valid = create_rocket_features(dls.valid, model)
X_train.shape, X_valid.shape

((3968, 20000), (768, 20000))

In [25]:
from sklearn.linear_model import RidgeClassifierCV
ridge = RidgeClassifierCV(alphas=np.logspace(-8, 8, 17), normalize=True)
ridge.fit(X_train, y_train)
print(f'alpha: {ridge.alpha_:.2E}  train: {ridge.score(X_train, y_train):.5f}  valid: {ridge.score(X_valid, y_valid):.5f}')

alpha: 1.00E-01  train: 0.99748  valid: 0.37500
